In [1]:
# %matplotlib inline
import pandas as pd
import plotly.express as px

import plotly.graph_objects as go
from ipywidgets import widgets
from ipywidgets import interact, interact_manual, Layout
from IPython.display import display
from IPython.display import display, clear_output

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sp
import matplotlib.dates as mdates


from jupyter_dash import JupyterDash
from dash import Dash, html, dcc


# Getting data in from CSVs to dataframes
ChildIdentifiers = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeChildIdentifiers.csv')
CINdetails = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeCINdetails.csv')
ChildCharacteristics = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeChildCharacteristics.csv')

def sample_graph(years, line_on):
    ChildIdentifiers = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeChildIdentifiers.csv')
    CINdetails = pd.read_csv('/workspaces/LA-Analyst-Tinkering/FAKE-CIN-DATA/FakeCINdetails.csv')

    merged_df = CINdetails.merge(ChildIdentifiers, left_on='LAchildID', right_on='LAchildID', how='left')


    df = merged_df[['LAchildID', 'PersonBirthDate', 'CINreferralDate', 'CINclosureDate']]

    df_dates = df.loc[:,['PersonBirthDate', 'CINreferralDate', 'CINclosureDate']].apply(
        pd.to_datetime, format='%Y/%m/%d', errors='coerce'
    )

    df_dates = df_dates[df_dates['CINclosureDate'].notna()]

    df_dates['CINrefToClose'] = (df_dates['CINclosureDate'] - df_dates['CINreferralDate'])
    df_dates['CINrefToClose'] = df_dates['CINrefToClose'].astype('timedelta64[h]')
    df_dates['CINrefToClose'] = df_dates['CINrefToClose'].astype(float) / 24
    
    df_dates = df_dates[(df_dates['PersonBirthDate'].dt.year >= years[0]) & (df_dates['PersonBirthDate'].dt.year <= years[1])]

    
    if line_on == 'Regression on':
        fig = px.scatter(df_dates, x='PersonBirthDate', y='CINrefToClose', trendline='ols',
            labels = {'PersonBirthDate' : 'Birth date',
                      'CINrefToClose': 'Wait between first referral and cin closure(days)'},
            title = ('Time between first referral and closure date by birth date'))
    else:
        fig = px.scatter(df_dates, x='PersonBirthDate', y='CINrefToClose',
            labels = {'PersonBirthDate' : 'Birth date',
                      'CINrefToClose': 'Wait between first referral and cin closure(days)'},
            title = ('Time between first referral and closure date by birth date'))
    fig.show()
   
    

year_select = widgets.IntRangeSlider(
    value = [2004, 2022],
    min = 2004,
    max = 2022,
    step = 1,
    description = 'Year Range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout = True,
    readout_format = 'd',
    layout={'width': '500px'}
    )

line_select = widgets.RadioButtons(
    options=['Regression on', 'Regression off',],
#     value='pineapple',
    description='Regression line',
    disabled=False
)


interact(sample_graph, years=year_select, line_on=line_select)


interactive(children=(IntRangeSlider(value=(2004, 2022), continuous_update=False, description='Year Range:', l…

<function __main__.sample_graph(years, line_on)>

Bad pipe message: %s [b'+ULN<\xa9a\xc3\xae\xc2W\xd1,\xb0\tz}\xfc 9d\x8es!HD~\xbe\xdeIx\x16.\xb4\x9f\xef\x86\xddojbm\x9b\tM\xac\xbf0O\x8b\x0c\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08', b'\n\x08\x0b\x08\x04\x08\x05\x08']
Bad pipe message: %s [b'\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'\x9c\xfem\xc4m\x9bR\xe8\'-[\xf5~|[0J\xa0 H<\xffL\\X\x1b\xbe\x0b\x9d\xa1\x85\xeaP\xbc\x7f\xc7\xd5##\xc9$\x81}v\xfe"\x11r\xd23\x9e\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c']
Bad pipe message: %s [b'\x95\x8a\xd9\xccZ\xd7\x12\xc2[/\r\xfd\x8d\xe0N\xc2)\xd6\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc